In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/IDSIA Biomedical Texts/AllSource_Intensity_ThirdJuly.csv', low_memory=False)
df.head(1)

,urls,text,source,label,WC,Analytic,Clout,Authentic,Tone,WPS,...,all_emo_labels,all_emo_label_rank,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,sadness_intensity,surprise_intensity,trust_intensity
0,https://www.quora.com/What-are-panic-attacks-l...,i have been dealing with these for quite some ...,Quora,1,607,55.22,35.35,48.82,1.0,26.39,...,"['fear', 'nervousness', 'confusion', 'curiosit...","{'fear': 1, 'nervousness': 2, 'confusion': 3, ...",0.415048,0.553423,0.272333,0.568205,0.4095,0.467625,0.4345,0.522773


## Getting 2 panic features

In [4]:
import regex as re

In [5]:
panic_symptoms = ["Palpitations", "Pounding heart", "Accelerated heart rate", "Sweating", "Trembling", "Shaking", "Shortness of breath",
"Smothering", "Feelings of choking", "Chest pain", "Discomfort", "Abdominal distress", "Nausea", "Dizziness", "Unsteadiness", "Lightheadedness",
"Faintness", "Chills", "Heat flashes", "Paresthesia", "Numbness", "Tingling sensations", "Derealization", "Depersonalization", "Fear of losing control",
"Fear of going crazy", "Fear of dying", "Mental images of dying", "Mental images of collapsing", "Agoraphobia", "Need to escape"]


panic_symptoms_ext = ["Palpitations", "Pounding heart", "Accelerated heart rate", "Sweating", "Trembling", "Shaking", "Shortness of breath",
"Smothering", "Feelings of choking", "Chest pain", "Discomfort", "Abdominal distress", "Nausea", "Dizziness", "Unsteadiness", "Lightheadedness",
"Faintness", "Chills", "Heat flashes", "Paresthesia", "Numbness", "Tingling sensations", "Derealization", "Depersonalization", "Fear of losing control",
"Fear of going crazy", "Fear of dying", "Mental images of dying", "Mental images of collapsing", "Agoraphobia", "Need to escape"
"Sweat", "Tremble", "Shake", "Shortage of breath","Feeling of choking","Dizzy",
"Faint","Fainted","Chill","Heat flash", "Numb", "Tingling sensation","Mental image of dying", "Mental image of collapsing"]

In [6]:
def count_symptoms_in_text(text, panic_list):
    if isinstance(text, str):
        count = 0
        for symptom in panic_list:
            match = re.search(r'\b{}\b'.format(symptom), text, re.IGNORECASE)
            if match:
                count += 1
        return count

In [7]:
df['symptoms_count'] = df['text'].apply(lambda x: count_symptoms_in_text(x,panic_symptoms))
df['symptoms_ext_count'] = df['text'].apply(lambda x: count_symptoms_in_text(x,panic_symptoms_ext))

In [8]:
df.head(1)

,urls,text,source,label,WC,Analytic,Clout,Authentic,Tone,WPS,...,anger_intensity,anticipation_intensity,disgust_intensity,fear_intensity,joy_intensity,sadness_intensity,surprise_intensity,trust_intensity,symptoms_count,symptoms_ext_count
0,https://www.quora.com/What-are-panic-attacks-l...,i have been dealing with these for quite some ...,Quora,1,607,55.22,35.35,48.82,1.0,26.39,...,0.415048,0.553423,0.272333,0.568205,0.4095,0.467625,0.4345,0.522773,8,8


In [9]:
pickle_in = open("/content/drive/MyDrive/IDSIA Biomedical Texts/Sentence Embeddings/AllSource_alldistilrobertav1_via_UMAP_SHORTembeddings.pickle", 'rb')
sentence_embeddings = pickle.load(pickle_in)
sentence_embeddings

array([[10.64194  ,  5.0430765,  5.6824026, ...,  4.9058275,  6.8707986,
         4.538727 ],
       [11.312859 ,  5.364349 ,  4.41365  , ...,  4.92234  ,  6.8475184,
         4.5590596],
       [10.531799 ,  4.894456 ,  5.387705 , ...,  4.8968716,  6.8360796,
         4.530069 ],
       ...,
       [10.346373 ,  4.4247556,  3.5815325, ...,  5.0401225,  6.552696 ,
         4.490976 ],
       [10.454275 ,  4.5640407,  3.6035635, ...,  5.0320673,  6.564847 ,
         4.4918733],
       [11.222271 ,  5.1468487,  4.0054016, ...,  5.079988 ,  6.6341186,
         4.5597043]], dtype=float32)

In [10]:
sentence_embeddings.shape

(7405, 28)

In [11]:
# Normalize sentence embeddings so that all values are between 0 and 1 (becasue emotions features are between 0 and 1 too)
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(0,1))  # (0,1) is default

In [12]:
sentence_embeddings = scaling.fit_transform(sentence_embeddings.reshape(-1, 1)).reshape(*sentence_embeddings.shape) # https://stackoverflow.com/questions/75461346/different-result-from-minmaxscaler-with-manual-calculations
sentence_embeddings

array([[0.8199141 , 0.35134387, 0.40484923, ..., 0.33985746, 0.5043063 ,
        0.30913472],
       [0.8760634 , 0.37823123, 0.298667  , ..., 0.3412394 , 0.502358  ,
        0.31083637],
       [0.81069636, 0.33890578, 0.3801859 , ..., 0.33910793, 0.5014007 ,
        0.30841014],
       ...,
       [0.795178  , 0.29959643, 0.22902691, ..., 0.35109666, 0.4776843 ,
        0.30513844],
       [0.80420834, 0.31125325, 0.2308707 , ..., 0.3504225 , 0.4787012 ,
        0.30521354],
       [0.8684821 , 0.3600286 , 0.26450062, ..., 0.354433  , 0.4844986 ,
        0.31089035]], dtype=float32)

In [13]:
sentence_embeddings.shape

(7405, 28)

In [14]:
# Standardization of LIWC features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [15]:
df.loc[:, 'WC':'Emoji']

,WC,Analytic,Clout,Authentic,Tone,WPS,BigWords,Dic,Linguistic,function,...,nonflu,filler,AllPunc,Period,Comma,QMark,Exclam,Apostro,OtherP,Emoji
0,607,55.22,35.35,48.82,1.00,26.39,25.86,93.74,68.86,57.50,...,0.0,0.0,19.93,4.61,9.88,0.16,0.16,2.80,2.31,0.33
1,401,54.08,1.31,99.00,8.57,13.83,14.21,90.02,74.81,61.60,...,0.0,0.0,13.47,6.98,4.74,0.25,0.00,1.50,0.00,0.00
2,446,25.83,93.36,75.79,1.00,12.74,15.47,95.74,75.34,59.64,...,0.0,0.0,16.59,8.07,2.69,0.22,0.00,4.71,0.90,0.00
3,525,30.63,2.97,96.06,6.86,15.44,12.38,90.86,73.33,56.57,...,1.9,0.0,21.52,5.33,4.95,1.14,0.00,6.67,3.43,0.00
4,323,21.57,1.00,99.00,1.12,13.46,15.48,93.50,75.85,60.99,...,0.0,0.0,18.27,7.12,4.33,0.93,0.00,5.26,0.62,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,237,5.24,20.52,53.23,15.51,16.93,18.57,93.25,81.86,62.87,...,0.0,0.0,10.13,5.49,1.69,0.00,0.00,2.53,0.42,0.00
7401,20,39.70,1.00,28.56,99.00,20.00,25.00,95.00,65.00,40.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7402,19,36.67,3.34,70.28,99.00,19.00,21.05,94.74,73.68,47.37,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7403,38,36.67,90.88,33.61,92.27,19.00,21.05,97.37,63.16,52.63,...,0.0,0.0,2.63,2.63,0.00,0.00,0.00,0.00,0.00,2.63


In [16]:
standardized_liwc = sc.fit_transform(df.loc[:, 'WC':'Emoji'])

In [17]:
standardized_liwc

array([[ 3.67802528,  0.98063234, -0.11115907, ...,  0.13908545,
        -0.12035728,  0.17485002],
       [ 2.12422261,  0.93614355, -1.01518881, ..., -0.32861383,
        -0.30797786, -0.05320853],
       [ 2.46364552, -0.16631989,  1.42946272, ...,  0.82624363,
        -0.23487893, -0.05320853],
       ...,
       [-0.75710077,  0.25671387, -0.96127635, ..., -0.86826685,
        -0.30797786, -0.05320853],
       [-0.61378887,  0.25671387,  1.36359921, ..., -0.86826685,
        -0.30797786,  1.76434898],
       [ 0.50253537, -0.1292459 ,  0.3546659 , ..., -0.47971667,
        -0.13335265, -0.05320853]])

In [18]:
# normalization of 2 panic features
normalized_panic_features = scaling.fit_transform(df[['symptoms_count', 'symptoms_ext_count']])

In [19]:
normalized_panic_features

array([[0.53333333, 0.47058824],
       [0.06666667, 0.05882353],
       [0.13333333, 0.17647059],
       ...,
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [20]:
normalized_panic_features.shape

(7405, 2)

## LR and XGB

In [21]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import roc_auc_score, make_scorer # not able to import balanced_accuray_score
from sklearn.model_selection import GridSearchCV # cross validation, used for hyperparamter tuning
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc

In [65]:
# run this for only intensity (8 features)
X = df.loc[:, 'anger_intensity':'trust_intensity'].values

# run this for Shortemb+Intensity (36 features)
# X = np.concatenate((df.loc[:, 'anger_intensity':'trust_intensity'] , sentence_embeddings), axis=1)

# run this for shortemb+intensity+stdliwc (154 features)
# temp = np.concatenate((df.loc[:, 'anger_intensity':'trust_intensity'] , sentence_embeddings), axis=1)
# X = np.concatenate((temp , standardized_liwc), axis=1)

# run this for shortemb+intensity+stdliwc+emotions(28) (182 features)
# temp1 = np.concatenate((df.loc[:, 'anger_intensity':'trust_intensity'] , sentence_embeddings), axis=1)
# temp2 = np.concatenate((df.loc[:, 'admiration':'neutral'] , standardized_liwc), axis=1)
# X = np.concatenate((temp1, temp2), axis=1)

# run this for shortemb+intensity+stdliwc+emotions whose intensity we have (Anger, disgust, fear, joy, sadness, surprise) (160 features)
# temp1 = np.concatenate((df.loc[:, 'anger_intensity':'trust_intensity'] , sentence_embeddings), axis=1)
# temp2 = np.concatenate((df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']] , standardized_liwc), axis=1)
# X = np.concatenate((temp1, temp2), axis=1)

# run this for shortemb+intensity+liwc+emotions(28)+normalized Panicfeatures (184 features)
# temp1 = np.concatenate((df.loc[:, 'anger_intensity':'trust_intensity'] , sentence_embeddings), axis=1)
# temp2 = np.concatenate((df.loc[:, 'admiration':'neutral'] , standardized_liwc), axis=1)
# temp3 = np.concatenate((temp1, temp2), axis=1)
# X = np.concatenate((temp3, normalized_panic_features), axis=1)

# run this for shortemb+intensity(6)+stdliwc+emotions whose intensity we have (Anger, disgust, fear, joy, sadness, surprise) (158 features)
# temp1 = np.concatenate((df[['anger_intensity', 'disgust_intensity', 'fear_intensity', 'joy_intensity', 'sadness_intensity', 'surprise_intensity']] , sentence_embeddings), axis=1)
# temp2 = np.concatenate((df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']] , standardized_liwc), axis=1)
# X = np.concatenate((temp1, temp2), axis=1)

# run this for shortemb+intensity(6)+stdliwc+emotions whose intensity we have (Anger, disgust, fear, joy, sadness, surprise) + Panic Extended (159 features)
temp1 = np.concatenate((df[['anger_intensity', 'disgust_intensity', 'fear_intensity', 'joy_intensity', 'sadness_intensity', 'surprise_intensity']] , sentence_embeddings), axis=1)
temp2 = np.concatenate((df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']] , standardized_liwc), axis=1)
temp3 = np.concatenate((temp1, temp2), axis=1)
X = np.concatenate((temp3, normalized_panic_features[:,1].reshape((-1, 1))), axis=1)

y = df['label']

In [70]:
X

array([[ 0.41504762,  0.27233333,  0.56820513, ..., -0.12035728,
         0.17485002,  0.47058824],
       [ 0.5304    ,  0.54125   ,  0.43216667, ..., -0.30797786,
        -0.05320853,  0.05882353],
       [ 0.4286    ,  0.22816667,  0.52619231, ..., -0.23487893,
        -0.05320853,  0.17647059],
       ...,
       [ 0.        ,  0.        ,  0.156     , ..., -0.30797786,
        -0.05320853,  0.        ],
       [ 0.344     ,  0.        ,  0.414     , ..., -0.30797786,
         1.76434898,  0.        ],
       [ 0.37675   ,  0.422     ,  0.51533333, ..., -0.13335265,
        -0.05320853,  0.        ]])

In [71]:
X.shape

(7405, 159)

In [72]:
clf = LogisticRegression(solver='lbfgs', max_iter=11000)
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=0, seed=42) # missing=None not working

In [73]:
logistic_f1scores = []
logistic_rocaucscore = []
logistic_praucscore = []
xgb_f1scores = []
xgb_rocaucscore = []
xgb_praucscore = []

for train_index, test_index in folds.split(X, y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    # Logistic
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred,average='weighted')
    logistic_f1scores.append(f1)
    rocaucscore = roc_auc_score(y_test, y_pred)
    logistic_rocaucscore.append(rocaucscore)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)  # Compute precision, recall, and thresholds
    pr_auc = auc(recall, precision)  # Calculate PR-AUC
    logistic_praucscore.append(pr_auc)

    # XGBoost
    clf_xgb.fit(X_train, y_train,
              verbose=False,  # set to True for individual use
              early_stopping_rounds=10,
              #eval_metrics='aucpr',
              eval_set = [(X_test,y_test)])
    y_pred = clf_xgb.predict(X_test)
    f1 = f1_score(y_test, y_pred,average='weighted')
    xgb_f1scores.append(f1)
    rocaucscore = roc_auc_score(y_test, y_pred)
    xgb_rocaucscore.append(rocaucscore)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)  # Compute precision, recall, and thresholds
    pr_auc = auc(recall, precision)  # Calculate PR-AUC
    xgb_praucscore.append(pr_auc)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

In [74]:
print(logistic_f1scores)
print(np.mean(logistic_f1scores))
print(np.std(logistic_f1scores))

[0.8776586272019143, 0.8631154744084146, 0.8833966351550845, 0.8713520103374509, 0.879176036225606, 0.857377106048732, 0.857377106048732, 0.8878787528904278, 0.8665954882834945, 0.8688705201287278]
0.8712797756728584
0.01003175729814911


In [75]:
print(logistic_rocaucscore)
print(np.mean(logistic_rocaucscore))
print(np.std(logistic_rocaucscore))

[0.8637339723639986, 0.8504255881986805, 0.8716816569152247, 0.8598009772189718, 0.8662664944603511, 0.8435469765117442, 0.8435469765117442, 0.8804660169915042, 0.8518240879560219, 0.8595077461269365]
0.8590800493255177
0.011402930134963954


In [76]:
print(logistic_praucscore)
print(np.mean(logistic_praucscore))
print(np.std(logistic_praucscore))

[0.9294817970170591, 0.9229256247382381, 0.9336249839179005, 0.9277103503813398, 0.9308158226751153, 0.9196672518321836, 0.9196672518321836, 0.938843299805354, 0.9237576036474305, 0.9280201224583873]
0.9274514108305191
0.005822658934671207


In [77]:
print(xgb_f1scores)
print(np.mean(xgb_f1scores))
print(np.std(xgb_f1scores))

[0.9080244974950559, 0.883029814310458, 0.9110576275262182, 0.9216697613058432, 0.9046736273883088, 0.8687719678968292, 0.9, 0.9138119069456728, 0.9129556067577823, 0.9190910174046414]
0.904308582703081
0.015743909432976176


In [78]:
print(xgb_rocaucscore)
print(np.mean(xgb_rocaucscore))
print(np.std(xgb_rocaucscore))

[0.8998078239760985, 0.8786062181003361, 0.9063278040582596, 0.9156759616581601, 0.8911054400597536, 0.8587737381309345, 0.8931003248375813, 0.9112162668665668, 0.9016741629185407, 0.915526611694153]
0.8971814352300382
0.016911744901851086


In [79]:
print(xgb_praucscore)
print(np.mean(xgb_praucscore))
print(np.std(xgb_praucscore))

[0.9481508042709631, 0.9383190988058268, 0.9522055647277575, 0.9565321323196198, 0.94300716013763, 0.9275770144544604, 0.9452586206896552, 0.9553936275254442, 0.9486874805840323, 0.95726096993696]
0.947239247345235
0.00876860918937835
